In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
X_context = np.load("X_context.npy")
X_seq = np.load("X_seq.npy")
y = np.load("y.npy")

In [3]:
X_seq

array([[[1.12370000e+04, 2.80000000e+02, 7.00000000e+00, 1.00000000e+02,
         0.00000000e+00, 1.20000000e+01],
        [9.97661504e+03, 2.45843996e+02, 6.91694725e+00, 0.00000000e+00,
         1.00000000e+00, 8.00000000e+00],
        [1.03103232e+04, 1.81702020e+02, 4.81313131e+00, 0.00000000e+00,
         1.00000000e+00, 8.00000000e+00],
        ...,
        [1.12885047e+04, 2.43702886e+02, 6.00000000e+00, 1.00000000e+02,
         0.00000000e+00, 1.20000000e+01],
        [1.15466442e+04, 2.58750842e+02, 6.25028058e+00, 1.00000000e+02,
         0.00000000e+00, 1.20000000e+01],
        [1.09620000e+04, 2.73000000e+02, 7.00000000e+00, 1.00000000e+02,
         0.00000000e+00, 1.20000000e+01]],

       [[1.06870000e+04, 2.32000000e+02, 6.00000000e+00, 9.90000000e+01,
         0.00000000e+00, 9.00000000e+00],
        [1.13356593e+04, 2.48772854e+02, 6.00000000e+00, 1.00000000e+02,
         0.00000000e+00, 9.00000000e+00],
        [1.07333008e+04, 2.60033317e+02, 6.00832931e+00, 1.000000

In [4]:
from sklearn.preprocessing import StandardScaler

In [5]:
scaler = StandardScaler()

In [6]:
X_seq_flat = X_seq.reshape(-1, X_seq.shape[-1])
X_seq_scaled = scaler.fit_transform(X_seq_flat)
X_seq = X_seq_scaled.reshape(X_seq.shape)

In [7]:
X_seq

array([[[ 0.80653561,  1.16983974,  1.07150979,  0.88477417,
         -0.53378254,  2.17124441],
        [-0.01911465,  0.68856125,  1.02652099, -1.55249831,
          2.02601213,  1.19631749],
        [ 0.19949019, -0.21523745, -0.11309382, -1.55249831,
          2.02601213,  1.19631749],
        ...,
        [ 0.84027517,  0.65839174,  0.52982038,  0.88477417,
         -0.53378254,  2.17124441],
        [ 1.00937668,  0.87042642,  0.66539472,  0.88477417,
         -0.53378254,  2.17124441],
        [ 0.6263892 ,  1.07120556,  1.07150979,  0.88477417,
         -0.53378254,  2.17124441]],

       [[ 0.4462428 ,  0.4934911 ,  0.52982038,  0.86040144,
         -0.53378254,  1.44004922],
        [ 0.87116513,  0.72983061,  0.52982038,  0.88477417,
         -0.53378254,  1.44004922],
        [ 0.47657344,  0.88849726,  0.53433227,  0.88477417,
         -0.53378254,  1.44004922],
        ...,
        [-0.10500782, -0.18604999, -0.01186903,  0.04343879,
         -0.53378254,  1.44004922],
  

In [8]:
X_context[0]

array([2.000e+00, 2.022e+03, 2.000e+00, 1.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
       0.000e+00, 1.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00,
       1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00])

In [9]:
column_to_normalize1 = X_context[:, 0].reshape(-1, 1)
column_to_normalize2 = X_context[:, 1].reshape(-1, 1)
column_to_normalize3 = X_context[:, 2].reshape(-1, 1)

In [10]:
standardized_column1 = scaler.fit_transform(column_to_normalize1)
standardized_column2 = scaler.fit_transform(column_to_normalize2)
standardized_column3 = scaler.fit_transform(column_to_normalize3)

In [11]:
X_context[:, 0] = standardized_column1.flatten()
X_context[:, 1] = standardized_column2.flatten()
X_context[:, 2] = standardized_column3.flatten()

In [12]:
X_context[0]

array([-1.53660388, -1.34478586, -0.95033935,  1.        ,  0.        ,
        0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ])

In [13]:
y = scaler.fit_transform(y.reshape(-1,1)).flatten()

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_seq_train, X_seq_test, X_context_train, X_context_test, y_train, y_test = train_test_split(X_seq, X_context, y, test_size=0.2, random_state=42)

In [16]:
import torch
from torch.utils.data import Dataset, DataLoader

In [17]:
class F1EmbedDataset(Dataset):
    def __init__(self, X_seq, X_context, y):
        self.X_seq = torch.tensor(X_seq, dtype=torch.float32)
        self.X_context = torch.tensor(X_context, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X_seq[idx], self.X_context[idx], self.y[idx]

In [18]:
train_dataset = F1EmbedDataset(X_seq_train, X_context_train, y_train)
test_dataset = F1EmbedDataset(X_seq_test, X_context_test, y_test)

In [19]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [20]:
import torch.nn as nn  
import torch.optim as optim  
import math

In [21]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
     
        # Initialize positional encoding matrix
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)
     
        # Calculate sinusoidal values for each dimension
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
     
        # Apply sine to even indices and cosine to odd indices
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
     
        # Add batch dimension
        self.register_buffer('pe', pe.unsqueeze(0))
    
    def forward(self, x):
        # Add positional encoding to embeddings
        x = x + self.pe[:, :x.size(1)].to(x.device)
        return x

In [22]:
class F1Embedder(nn.Module):
    def __init__(self, telemetry_dim, context_dim, d_model=512, nhead=4, num_layers=2):
        super().__init__()
        self.embedding_layer = nn.Linear(telemetry_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model)

        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.context_mlp = nn.Sequential(
            nn.Linear(context_dim, 64),
            nn.ReLU(),
            nn.Linear(64, d_model)
        )

        self.regressor = nn.Sequential(
            nn.Linear(d_model * 2, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x_seq, x_context):
        x_seq = self.embedding_layer(x_seq)
        x_seq = self.pos_encoder(x_seq)
        z_seq = self.transformer(x_seq)
        z_seq = z_seq.mean(dim=1)

        z_context = self.context_mlp(x_context)

        combined = torch.cat([z_seq, z_context], dim=1)
        output = self.regressor(combined)
        return output.squeeze()

In [23]:
model = F1Embedder(
    telemetry_dim=X_seq.shape[2],
    context_dim=X_context.shape[1],
    d_model=64
)

In [24]:
print(len(X_seq_train), len(X_context_train), len(y_test))

1416 1416 354


In [25]:
import torch.optim as optim
import torch.nn as nn

optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

In [26]:
from tqdm import tqdm

In [27]:
model.train()

F1Embedder(
  (embedding_layer): Linear(in_features=6, out_features=64, bias=True)
  (pos_encoder): PositionalEncoding()
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (context_mlp): Sequential(
    (0): Linear(in_features=22, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
  )
  (re

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

for X_seq, X_context, y in tqdm(train_loader, desc="training"):
    X_seq = X_seq.to(device)
    X_context = X_context.to(device)
    y = y.to(device)

    optimizer.zero_grad()

    pred = model(X_seq, X_context)       # Forward pass
    loss = criterion(pred, y)            # Compute loss

    loss.backward()                      # Backward pass
    optimizer.step()                     # Update weights

device: cpu


training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:12<00:00,  3.56it/s]


In [29]:
model.eval()

F1Embedder(
  (embedding_layer): Linear(in_features=6, out_features=64, bias=True)
  (pos_encoder): PositionalEncoding()
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (context_mlp): Sequential(
    (0): Linear(in_features=22, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
  )
  (re

In [30]:
all_preds = []
all_targets = []

with torch.no_grad():
    for X_seq, X_context, y in test_loader:
        X_seq = X_seq.to(device)
        X_context = X_context.to(device)
        y = y.to(device)

        preds = model(X_seq, X_context)
        all_preds.append(preds.cpu())
        all_targets.append(y.cpu())

# Stack all batches into single arrays
all_preds = torch.cat(all_preds).numpy()
all_targets = torch.cat(all_targets).numpy()

In [31]:
y_pred_orig = scaler.inverse_transform(all_preds.reshape(-1, 1))
y_test_orig = scaler.inverse_transform(all_targets.reshape(-1, 1))

In [32]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

def mean_squared_error(a, b):
    mse = np.mean(np.square(a - b))
    return mse
    
mse = mean_squared_error(y_test_orig, y_pred_orig)
mae = mean_absolute_error(y_test_orig, y_pred_orig)
r2 = r2_score(y_test_orig, y_pred_orig)

print(f"Test MSE: {mse:.4f}")
print(f"Test MAE: {mae:.4f}")
print(f"R2 Score: {r2:.4f}")

Test MSE: 14.8579
Test MAE: 2.7879
R2 Score: 0.8235


In [34]:
torch.save(model, 'f1-embed-model.pt')